In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import os
import copy
from torchvision import datasets, models, transforms

import torch.utils.data as utils
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import time
from torch.optim import lr_scheduler
from PIL import Image
import PIL.ImageOps    
import torch.nn.functional as F
import pandas as pd 

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.0.1
Torchvision Version:  0.2.1


In [2]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for data in dataloaders[phase]:
                img, label = data
                img ,label = img.to(device) , label.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    res = model(img)
                    res = res.to(device)
                    label=label.float()
                    label=label.flatten()
                    loss = criterion(res, label)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # statistics
                running_loss += loss.item()
                if torch.abs(torch.sum(res.data - label.data))/128 < 10.0:
                    running_corrects += 1
                    
#                 print("model:", end=''); print(torch.mean(res.data))
#                 print("loss: ", end=''); print(torch.abs(torch.sum(res.data - label.data))/128)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [3]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [4]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [5]:
class DepressingNet(nn.Module):
    
    def __init__(self, model):
        super(DepressingNet, self).__init__()
        self.model = model
        self.output_layer = nn.Sequential(nn.Linear(50,1))

    def forward(self, inp):
        out = self.model(inp)
        res = self.output_layer(out)
#         res = torch.clamp(out,min=11.0, max=50.0)
        return res
    

In [6]:
class DepressingNetDataset():
    
    def __init__(self,training_csv=None,training_dir=None,transform=None):
        self.training_df=pd.read_csv(training_csv)
        self.training_df.dropna(inplace=True)
        self.training_dir = training_dir    
        self.transform = transform

    def __getitem__(self,index):
        image_path=os.path.join(self.training_dir,str(self.training_df['image'].iloc[index])+'.jpg')  
        img = Image.open(image_path)       
        # Apply image transformations
        if self.transform is not None:
            img = self.transform(img)
        
        return img, torch.from_numpy(np.array([float(self.training_df['mean_rank'].iloc[index])],dtype=np.float32))
    
    def __len__(self):
        return len(self.training_df)

In [7]:
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

class Config():
    training_csv = "/home/bill/urban_planning/ranked_depressing_train.csv"
    testing_csv ="/home/bill/urban_planning/ranked_depressing_test.csv"
    image_dir="/home/bill/urban_planning/images_only/"
    train_batch_size = 32
    train_number_epochs = 101
    num_classes=50



In [8]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
config = Config()
model_name = "vgg"
num_classes = config.num_classes
batch_size = config.train_batch_size
num_epochs = config.train_number_epochs
feature_extract = True
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

In [9]:
train_dataset = DepressingNetDataset(config.training_csv,config.image_dir,
                                        transform=transforms.Compose([transforms.Resize((input_size, input_size)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                                      ])
                                       )

test_dataset = DepressingNetDataset(config.testing_csv,config.image_dir,
                                        transform=transforms.Compose([transforms.Resize((input_size, input_size)),
                                        transforms.CenterCrop(input_size),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                                      ])
                                       )

In [10]:
train_dataloader = DataLoader(train_dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=config.train_batch_size)

test_dataloader = DataLoader(test_dataset,
                             num_workers=6,
                             batch_size=config.train_batch_size,
                             shuffle=True)

dataloaders_dict={'train':train_dataloader, 'val':test_dataloader}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
model = DepressingNet(model_ft)
model = model.to(device)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [12]:
criterion = nn.MSELoss()
model, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/100
----------
train Loss: 0.5409 Acc: 0.0260
val Loss: 0.4069 Acc: 0.0140
Epoch 1/100
----------
train Loss: 0.4391 Acc: 0.0268
val Loss: 0.3596 Acc: 0.0276
Epoch 2/100
----------
train Loss: 0.4082 Acc: 0.0269
val Loss: 0.4175 Acc: 0.0062
Epoch 3/100
----------
train Loss: 0.3797 Acc: 0.0280
val Loss: 0.3277 Acc: 0.0282
Epoch 4/100
----------
train Loss: 0.3628 Acc: 0.0279
val Loss: 0.3245 Acc: 0.0268
Epoch 5/100
----------
train Loss: 0.3492 Acc: 0.0277
val Loss: 0.3430 Acc: 0.0185
Epoch 6/100
----------
train Loss: 0.3369 Acc: 0.0283
val Loss: 0.3532 Acc: 0.0142
Epoch 7/100
----------
train Loss: 0.3268 Acc: 0.0287
val Loss: 0.3161 Acc: 0.0248
Epoch 8/100
----------
train Loss: 0.3210 Acc: 0.0283
val Loss: 0.3016 Acc: 0.0297
Epoch 9/100
----------
train Loss: 0.3163 Acc: 0.0284
val Loss: 0.2997 Acc: 0.0296
Epoch 10/100
----------
train Loss: 0.3113 Acc: 0.0287
val Loss: 0.2959 Acc: 0.0300
Epoch 11/100
----------
train Loss: 0.3092 Acc: 0.0282
val Loss: 0.3199 Acc: 0.0221
Ep

val Loss: 0.2998 Acc: 0.0275
Epoch 98/100
----------
train Loss: 0.2973 Acc: 0.0287
val Loss: 0.2971 Acc: 0.0279
Epoch 99/100
----------
train Loss: 0.2968 Acc: 0.0286
val Loss: 0.2932 Acc: 0.0294
Epoch 100/100
----------
train Loss: 0.2966 Acc: 0.0288
val Loss: 0.3097 Acc: 0.0241
Training complete in 570m 18s
Best val Acc: 0.030959


In [13]:
hist

[0.014014524143202955,
 0.027646833991591284,
 0.006242833481972226,
 0.028220155433813225,
 0.026818703019492927,
 0.01847369091604026,
 0.014205631290610268,
 0.02478022678048159,
 0.02974901261307173,
 0.02962160784813352,
 0.030003822142948146,
 0.022104726716779207,
 0.029685310230602623,
 0.011466428844438782,
 0.010956809784685948,
 0.02917569117084979,
 0.02872977449356606,
 0.029685310230602623,
 0.02261434577653204,
 0.029366798318257104,
 0.028347560198751433,
 0.030577143585170087,
 0.025863167282456363,
 0.02962160784813352,
 0.02331507198369219,
 0.030831953115046502,
 0.030704548350108295,
 0.030258631672824565,
 0.03019492929035546,
 0.030386036437762772,
 0.029876417378009938,
 0.030449738820231876,
 0.026882405401962035,
 0.03032233405529367,
 0.028538667346158748,
 0.03064084596763919,
 0.029303095935787997,
 0.029557905465664416,
 0.030003822142948146,
 0.03032233405529367,
 0.030831953115046502,
 0.019811440947891452,
 0.030386036437762772,
 0.030386036437762772,
 

In [14]:
torch.save(model.state_dict(), "0_3lossmodel.pt")
print("Model Saved Successfully")

Model Saved Successfully


In [17]:
img,lab=train_dataset.__getitem__(0)
data

(tensor([[[-0.0801, -0.0972, -0.0801,  ...,  1.5810,  1.5982,  1.6153],
          [-0.0458, -0.0629, -0.0458,  ...,  1.5810,  1.5982,  1.6153],
          [ 0.0398,  0.0227,  0.0227,  ...,  1.5982,  1.6153,  1.6324],
          ...,
          [-1.5699, -1.4158, -1.4672,  ...,  1.2557,  1.1187,  0.9988],
          [-1.5870, -1.5870, -1.5870,  ...,  1.3242,  1.1872,  1.0673],
          [-1.5870, -1.5870, -1.5870,  ...,  1.3927,  1.2385,  1.1187]],
 
         [[-0.0224, -0.0399, -0.0399,  ...,  1.8158,  1.8333,  1.8508],
          [ 0.0126, -0.0049, -0.0049,  ...,  1.8158,  1.8333,  1.8508],
          [ 0.1001,  0.0826,  0.0826,  ...,  1.8333,  1.8508,  1.8683],
          ...,
          [-1.4930, -1.3354, -1.3880,  ...,  1.6057,  1.4832,  1.3782],
          [-1.5105, -1.5105, -1.5105,  ...,  1.6758,  1.5532,  1.4482],
          [-1.5105, -1.5105, -1.5105,  ...,  1.7458,  1.6232,  1.5007]],
 
         [[-0.0092, -0.0267, -0.0267,  ...,  2.0474,  2.0648,  2.0823],
          [ 0.0256,  0.0082,

In [22]:
model(img.unsqueeze(0).to('cuda:0'))

tensor([[24.9208]], device='cuda:0', grad_fn=<AddmmBackward>)

In [19]:
lab

tensor([27.7299])